# Text Summarizer:  Producing a concise and fluent summary while preserving key information and overall meaning

Extractive Summarization attempts to summarize articles by selecting a subset of words that retain the most important points. This approach weights the important part of sentences and uses the same to form the summary.
Sentences are weighted and ranked based on importance and similarity among each other. Cosine similarity is primarily used to measure similarity.

In [13]:
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import codecs

In [14]:
def read_article(text):
    # file = codecs.open(file_name, "r", encoding='utf-8')  # handles accentuated characters
    # filedata = file.readlines()
    article = text.split(". ")    # split the text by sentences using ". "
    
    sentences = []
    for sentence in article:             # iterate thru sentences, printing each and generate list of wards for each sentence
        #print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))    # replace any non character by " "
    #sentences.pop()   ##### systematically eliminate last sentence of the text from the returned sentences??
    
    return sentences

In [15]:
def sentence_similarity(sentence_1, sentence_2, stopwords=None):
    if stopwords is None:
        stopwords = []     # create an empty list to avoid error below
 
    sentence_1 = [w.lower() for w in sentence_1]
    sentence_2 = [w.lower() for w in sentence_2]

    all_words = list(set(sentence_1 + sentence_2))  # create total vocabulary of unique words for the two sentences compared

    vector1 = [0] * len(all_words)                  # prepare one-hot vectors for each sentence over all vocab
    vector2 = [0] * len(all_words)

    # build the vector for the first sentence
    for w in sentence_1:
        if w in stopwords:
            continue 
        vector1[all_words.index(w)] += 1           # list.index(element) returns the index of the given element in the list

    # build the vector for the second sentence
    for w in sentence_2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1

    return 1 - cosine_distance(vector1, vector2)   # Cosine = 0 for similar sentences => returns 1 if perfectly similar

In [16]:
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))  # create a square matrix with dim the num of sentences
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences (diagonal of the square matrix)
                continue
            # similarity of each sentence to all other sentences in the text is measured and logged in the matrix
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

In [17]:
def generate_summary(text, top_n=5, show=False):
    #stop_words = stopwords.words('english')
    stop_words = stopwords.words('indonesian')
    summarize_text = []
    
    # Step 1 - Read text and tokenize
    sentences =  read_article(text)
    print("number of sentences in text : ", len(sentences))
    
    # Step 2 - Generate Similary Matrix across sentences
    sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words)
    
    # Step 3 - Rank sentences in similarity matrix. let’s convert the similarity matrix into a graph. 
    # The nodes of this graph will represent the sentences and the edges will represent the similarity scores between
    # the sentences. On this graph, we will apply the PageRank algorithm to arrive at the sentence rankings.
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)
    
    # Step 4 - Sort the rank and pick top sentences extract the top N sentences based on their rankings for summary generation
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    if show :
        print("Indexes of top ranked_sentence order are ", ranked_sentence)
    # extract the top N sentences based on their rankings for summary generation
    for i in range(top_n):
        summarize_text.append(" ".join(ranked_sentence[i][1]))
    
    # Step 5 - Output the summarize text
    print("Summarize Text: \n", ". ".join(summarize_text)+'.')

In [18]:
# let's begin
generate_summary( "   bab 1 pendahuluan 1.1 latar belakang seiring berkembangnya teknologi di indonesia, banyak entrepreneur yang  memanfaatkan  kemajuan  ini  sebagai  sarana  bisnis  dengan  basis online. didukung juga dengan masuknya industri 4.0. untuk itu, sektor industri  nasional  perlu  banyak  pembenahan  terutama  dalam  aspek penguasaan  teknologi  yang  menjadi  kunci  penentu  daya  saing  di  era industri 4.0. adapun lima teknologi utama yang menopang pembangunan sistem  industry  4.0,  yaitu  internet  of  things,  artificial  intelligence, human–machine interface, teknologi robotik dan sensor, serta teknologi 3d  printing  (“kemenperin:  making  indonesia  4.0:  strategi  ri  masuki revolusi industri  ke-4,”  2019).  pemerintah juga  mendukung perkembangan  internet  dengan  membangun  infrastruktur-infrastruktur untuk perkembangan teknologi digital, seperti pembangunan palapa ring yaitu  pembangunan  serat  optik  di  seluruh  indonesia  sepanjang  36.000 kilometer (pdsi kominfo, 2019). seperti  namanya,  cms  atau  content  management  system  dimana kegunaanya untuk membantu pelaku bisnis dalam manajemen bisnis agar lebih terstruktur. toko offline pelaku bisnis membutuhkan sebuah sistem point of sale, atau sistem kasir, pos (point of sale) adalah istilah yang dapat  mencakup  alat  bantu  dagang,  display,  dan  perangkat  untuk melakukan  transaksi  seperti  mesin  kasir  dan  komputer.  yang  dapat membantu dalam melayani pelanggan secara face-to-face. dalam toko  sembako  grosir  yang  masih  menggunakan  cara konvensional,  setiap  transaksi  dilakukan  secara  manual,  akan  sulit mempertahankan  integritas  data  keuangan  dan  informasi  stok.  tentunya akan menjadi sangat rentan terhadap manipulasi data. usaha konvensional tidak memiliki sistem yang teratur untuk mendukung operasi sehari-hari, sebagai  contoh,  ketika  pelanggan  ingin  membeli  barang  dari  toko, karyawan toko tidak memiliki gambaran yang jelas terhadap produk yang tersedia di toko alhasil karyawan toko tidak dapat memberikan informasi yang tepat untuk pelanggan karena tidak ada 1  2 suatu source of truth atau sumber kebenaran yang dapat diandalkan. selain  itu  banyak  terjadi  kehilangan  barang  pada  proses  pengecekan barang  sebelum  diberikan  ke  customer  sehingga  pemilik  toko  sembako grosir mengalami kerugian. dan  terkadang  beberapa  customer  toko  sembako  grosir  kurang membawa uang dan cashier toko sembako grosir pun lupa untuk mencatat hutang  tersebut  di  buku  hutang  sehingga  menyebabkan  data  keuangan menjadi tidak sesuai dan keliru. dari sini mulai ada beberapa pertanyaan muncul seperti bagaimana jika online dan offline ini dapat dijalankan secara bersamaan. terdapat masalah di  mana  karyawan  toko  sembako  salah  melakukan  perhitungan  stok barang, salah pencatatan barang yang telah dikembalikan dan lain-lain. dengan  mempertimbangkan  permasalahan  di  atas,  maka  penulis ingin  mengembangkan  pembuatan  aplikasi  yang  dapat  membantu  para pemilik  toko  sembako  grosir  dengan  skala  kecil  sehingga  dapat  ikut bersaing. aplikasi ini diharapkan dapat mempermudah para pemilik toko sembako grosir untuk mengembangkan bisnisnya dengan adanya efisiensi waktu  dan  terstrukturnya  sistem manajemen  dalam  suatu  usaha terutama bagi para pemilik toko sembako grosir yang selama ini telah menerapkan proses jual beli secara konvensional. rumusan masalah yang didapat berdasarkan latar belakang di atas adalah 1.  bagaimana  tidak  efisiennya  dalam  bertransaksi  jual  beli  dalam operasional toko sembako grosir? 2.  bagaimana diketemukannya kesalahan yang muncul sehingga bisa menyebabkan  produk  toko  hilang  dan  stok  tidak  terdata  dengan 3.  bagaimana terjadinya kehilangan barang dalam toko sembako grosir sehingga semua stok barang dapat terdata dengan baik? 4.  bagaimana  tidak  teraturnya  alur  operasi  toko  sembako  karena dilakukan secara manual sekaligus tidak punya standar operasi yang benar. baik ?  1.2  rumusan masalah sebagai berikut:       3 1.3  hipotesis berdasarkan  kajian  teori  dan  kerangka  pemikiran,  peneliti  menyatakan hipotesis penelitian dari penelitian ini adalah: 1.  berdasarkan wawancara pengguna yang telah kami lakukan, metode operasi  toko  sembako  grosir  yang  konvensional  di  mana  setiap transaksi  dicatat  secara  manual  yang  rentan  dengan  human  error, contohnya,  salah  memasukkan  stok  barang,  nota  penjualan,  yang mengakibatkan laporan keuangan yang tidak akurat. 2. karena alur operasi toko sembako tidak mempunya standar prosedur yang efisien, pengguna juga berpendapat bahwa apabila ada aplikasi yang  bisa  mengatur  toko  dengan  alur  operasi  yang  jelas  akan mempermudah kegiatan sehari-hari toko, menjaga integritas data stok, transaksi,  dan  lain-lain  akan  sangat  membantu  untuk  menghindari human error. 1.4  ruang lingkup mencakup : aplikasi  belanjayuk.id  akan  dibagi  menjadi  tiga  bagian  yang  berbeda, 1.  back-office,  adalah  aplikasi  yang  berbasis  web  yang  digunakan  oleh pemilik  toko  untuk  monitor  statistik  inventaris,  penjualan,  dan  lainnya, yang  bertujuan  untuk memberikan  ikhtisar  untuk  pemilik  usaha tersebut dan  juga  mempersingkat  serta  memudahkan  pemilik  usaha  untuk melakukan pemeriksaan terhadap perkembangan usahanya. 2.  sistem  kasir,  adalah  aplikasi  yang  berbasis  web,  untuk  melakukan transaksi penjualan toko. fitur yang dihadirkan untuk sistem kasir hampir sama dengan yang dimiliki admin. 3.  aplikasi checker, adalah aplikasi yang digunakan oleh karyawan untuk memeriksa barang-barang baik yang akan di antar ataupun diambil secara langsung  oleh  pembeli  serta  memastikan  barang-barang  yang  diterima pembeli dalam kondisi baik. penulis berharap dengan adanya sistem ini, dapat membantu       4 mengubah metode operasi toko sembako grosir yang konvensional di mana setiap  transaksi  dicatat  secara  manual  yang  rentan  dengan  human  error,. semua  aplikasi  akan  dibuat  dengan  menggunakan  bahasa  indonesia, mengingat target yang dituju berada dalam skala menengah ke bawah. 1.5  tujuan dan manfaat 1.5.1  tujuan tujuan dari penelitian ini adalah: 1.  proses bertransaksi jual beli dalam operasional toko sembako grosir menjadi lebih effisien 2.  membantu  pemiliki  toko  untuk  meminimalisir  hilangnya produk atau stok akibat pendataan manual. dalam penelitian disebut pos (point of sales). 3.  menghasilkan suatu aplikasi yang dapat membantu proses pengecekan  barang  agar  barang  yang  diberikan  ke customer sesuai dengan yang dibeli dan memberikan alur operasi toko yang jelas. dalam penelitian disebut checker apps. 4.  menghasilkan  suatu  aplikasi  yang  dapat  mengurangi human  error  dan  mempermudah  pemilik  toko  sembako grosir  dalam  mengelola  keseluruhan  sistem  usahanya. dengan  memberikan suatu sistem  digital  yang memungkinkan pemilik toko untuk mendapatkan statistik toko.  dalam  penelitian  disebut  cms (content management system) 1.5.2  manfaat manfaat yang diharapkan jika aplikasi ini berjalan adalah: 1.  dengan adanya aplikasi ini diharapkan dapat mengurangi kelalaian atau human error dari karyawan dalam melayani operasional  menjadi  lebih  efisien  dengan  adanya  alur operasional yang jelas.     5 2.  dengan  adanya  aplikasi  ini  dapat  membantu  pendataan transaksi  penjualan  barang  secara  digital  dan  dapat mempermudah untuk membuat laporan penjualan 3.  meminimalisir  kehilangan  barang  karena  kesalahan  saat barang  diberikan  kepada  pelanggan  seperti  barang  yang diberikan pelanggan lebih dari barang yang dibeli. dan juga membantu agar barang yang diberikan pada customer sesuai dengan yang dibeli. 4.  dengan  adanya  aplikasi  ini  dapat  membantu  dalam manajemen  keseluruhan  toko  agar  lebih  terstruktur dan  pemilik  toko  dapat  mendapatkan  statistic  toko sembako  dalam  satu  source  of  truth.  dan  juga membantu pembeli melakukan transaksi dengan cepat karena adanya standar operasi yang efisien. 1.6  metode penelitian penelitian ini menggunakan 2 metode, yaitu : metode analisis dan metode perancangan.  pada  metode  analisis,  pengumpulan  data  dilakukan  secara kualitatif,  dimana  dapat  dilakukan  selama  proses  wawancara  berlangsung. sedangkan  untuk  dapat  membantu,  mengatur,  mengontrol  dan  mengevaluasi sistem yang dirancang, metode perancangan dibagi menjadi 2 yaitu: 1.  software design document pada  proses  membangun  sistem  dibutuhkan  dokumen-dokumen perancangan  aplikasi  seperti  deskripsi  software,  fungsi-fungsi software, dan kebutuhan teknologi. 2.  perancangan sistem pada proses perancangan sistem dibagi menjadi 3 bagian yaitu: a. pendekatan ooad dalam pengembangan sebuah aplikasi diperlukan alur-alur aplikasi  dan  gambaran  dasar tentang  aplikasi.  pada pendekatan  ooad  dilakukan  dengan  tahapan  pembuatan diagram  use  case  diagram,  use  case  description,  class 6    diagram, activity diagram, dan sequence diagram. b. perancangan user interface dalam  perancangan  user interface  menggunakan wireframe  design  yang  berisi  gambaran  visual  yang  akan dikembangkan untuk aplikasi ini. c. perancangan database dalam  tahapan  perancangan  database  atau  basis  data menggunakan entity relationship diagram (erd). 1.7  sistematika penulisan sistematika penyusunan laporan dideskripsikan berdasarkan bagian-bagian, sebagai berikut: bab 1 pendahulan pada  bab  ini,  menjelaskan  tentang  latar  belakang,  rumusan masalah,  hipotesis,  ruang  lingkup,  tujuan  dan  manfaat,  metode penelitian, dan sistematika dalam penulisan laporan. bab 2 tinjauan referensi pada  bab  ini,  menjelaskan  teori  yang  digunakan  dalam penelitian,  yaitu  teori  umum  dan  teori  khusus.  teori-teori  umum meliputi : pos (point of sales), cms (content management system), oop (object oriented programming), mv* pattern, five mesurable human  factors,  uml  (unified  modeling  language),  waterfall software  development,  use  case  diagram,  use  case  description, activity  diagram,  class  diagram,  sequence  diagram,  basis  data, entity relationship diagram, dependency injection, cross-platform software, functional relational mapper, continuous integration and continuous delivery, paas (platform as a service), version control system  dan  git,  dan  sql  (structured  query  language).  dan  teori- teori khusus meliputi: json (javascript object notation), scala, vue, postgresql,  declarative  user  interface  dan  qr  code  (quick response code). bab 3 metode penelitian    7 pada  bab  ini,  menjelaskan  metode  yang  digunakan  dalam pengembangan  aplikasi,  yaitu  metode  penelitian,  analisis,  dan perancangan.  metode penelitian yang digunakan untuk mengembangkan  aplikasi  adalah  metodologi  waterfall  software development. sedangkan untuk analisis meliputi: analisis sistem yang berjalan  atau  perbandingan  dengan  aplikasi  sejenis,  analisis permasalahan/kebutuhan, dan usulan pemecahan masalah. dan dalam perancangan aplikasi meliputi: usulan pemecahan masalah yang berisi deskripsi  software,  fungsi-fungsi  software  dan  kebutuhan  teknologi, perancangan sistem yang berisi pendekatan ooad (object oriented analysis and design), perancangan user interface, dan perancangan database. bab 4 hasil dan pembahasan pada bab ini, menjelaskan tentang hasil dan pembahasan yang meliputi:  spesifikasi  sistem,  prosedur  penggunaan  aplikasi  meliputi backoffice,  cashier  dan  checker  apps,  dan  evaluasi  yang  meliputi evaluasi user interface dan evaluasi sistem. bab 5 kesimpulan dan saran pada bab ini, menjelaskan kesimpulan dari penelitian ini dan saran untuk pengembangan aplikasi ini. 8  ", 5, False)

number of sentences in text :  88
Summarize Text: 
 bab 1 pendahuluan bab  ini  berisi  mengenai  latar  belakang  masalah,  rumusan  masalah, hipotesis,  ruang  lingkup,  tujuan  dan  manfaat  dari  skripsi,  metode  penelitian yang  digunakan  serta  sistematika  penulisan  yang  menggambarkan  isi  dari skripsi ini.  aplikasi  e-learning  seperti  apa  yang  perlu  dikembangkan  untuk pembelajaran  pemrograman  berbasis  obyek  dan  basis  data  bagi  novice 2.  memudahkan  pengajar  dalam  mengajar  materi  pemrograman  berbasis 3.  aplikasi  diperuntukkan  bagi  programmer  pemula  (novice  programmer) yang ingin  memulai  atau sedang  melaksanakan  pembelajaran pemrograman berorientasi obyek dan basis data. bab 3 metode penelitian bab  ini  berisi  mengenai  analisis  masalah-masalah  yang  ada  dan langkah-langah  penyelesaian  masalah tersebut.


In [177]:
generate_summary( "dgse.txt", 3)

number of sentences in text :  22
Summarize Text: 
 Le 28 juillet, les trois hommes sont mis en examen pour «tentative d'homicide volontaire en bande organisée», «recel en bande organisée de vol, transport, acquisition, détention d'armes de catégorie B en réunion» et «association de malfaiteurs en vue de la commission de crimes et délits punis de 10 ans d'emprisonnement». Selon le parquet de Paris, les deux jeunes militaires arrêtés le 24 juillet à Créteil (Val-de-Marne) semblaient viser une femme de 54 ans. Les 30 et 31 juillet, deux autres hommes sont à leur tour placés en garde à vue.


In [178]:
generate_summary( "dreyfus.txt", 3)

number of sentences in text :  21
Summarize Text: 
 Malgré les menées de l'armée pour étouffer cette affaire, le premier jugement condamnant Dreyfus est cassé par la Cour de cassation au terme d'une enquête minutieuse, et un nouveau conseil de guerre a lieu à Rennes en 1899. À cette date, l'opinion comme la classe politique française est unanimement défavorable à Dreyfus. Le même mois, Mathieu Dreyfus porte plainte auprès du ministère de la Guerre contre Walsin Esterhazy.


In [179]:
generate_summary( "summarize.txt", 3)

number of sentences in text :  11
Summarize Text: 
 The limited study is available for abstractive summarization as it requires a deeper understanding of the text as compared to the extractive approach. It’s good to understand Cosine similarity to make the best use of code you are going to see. Since we will be representing our sentences as the bunch of vectors, we can use it to find the similarity among sentences.


In [190]:
generate_summary( "maria.txt", 5)

number of sentences in text :  16
Summarize Text: 
 So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think everyone just thinks because we're tennis players we should be the greatest of friends. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net. I think just because you're in the same sport doesn't mean that you have to be friends with everyone just because you're categorized, you're a tennis player, so you're going to get along with tennis players.
